In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications import MobileNetV2
import numpy as np
import pandas as pd
import os
import zipfile

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
with zipfile.ZipFile('Data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    '/content/Data/Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training')
validation_generator = train_datagen.flow_from_directory(
    '/content/Data/Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation')

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/Data/TestDataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy}')
print(f'Test loss: {test_loss}')

In [ ]:
# Caminho da imagem a testar
img_path = '/content/Data/TestDataset/Jim Carrey/91.jpg'

img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

prediction = model.predict(img_array)
print('Prediction:', prediction)

if prediction >= 0.5:
    print("A imagem é do Tom Hanks")
else:
    print("A imagem é do Jim Carrey")